In [ ]:
import numpy as np

In [ ]:
x = np.random.random((10,3))   # Task 1
print(x)
print([np.where(abs(0.5-elem)==np.min(abs(0.5-elem)))[0] for elem in x])

In [ ]:
x = np.random.random((6,6))   # Task 2
print(x)
print([np.sum(x[i])/(np.min([x[j][i] for j in range(6)])) for i in range(6)])

In [ ]:
x = np.array([6, 2, 0, 3, 0, 0, 5, 7, 0])   # Task 3
print(np.max([x[np.where(x==0)[0][i]+1] for i in range(np.sum(x==0)-np.sum(x[len(x)-1]==0))]))

In [ ]:
x = np.ones(10)               # Task 4
i = np.array([0, 1, 2, 3, 5, 5, 5, 8])
print([x[j]+np.sum(i==j) for j in range(10)])

In [ ]:
url = 'https://raw.githubusercontent.com/sharmaroshan/Clustering-of-Mall-Customers/master/Mall_Customers.csv'
data = np.loadtxt(url, delimiter=',', dtype=object, skiprows=1)
data[:5]

In [ ]:
data[:, 1] = data[:, 1] == 'Male'
data = data.astype(float)
data[:5]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(data[:, 2], data[:, 3])
plt.xlabel('Age')
plt.ylabel('Annual Income')
plt.show()

In [ ]:
plt.scatter(data[:, 2], data[:, 4])
plt.xlabel('Age')
plt.ylabel('Spending Score')
plt.show()

In [ ]:
plt.scatter(data[:, 3], data[:, 4])
plt.xlabel('Annual Income')
plt.ylabel('Spending Score')
plt.show()

In [ ]:
def standardization(data):   # Нормализация данных
    mean = data.mean(axis=0)
    std = data.std(axis=0)
    return (data - mean)/std

In [ ]:
import numpy.random 
from scipy.spatial import distance

In [ ]:
def kmeans(obs, k_or_guess, iter=20, thresh=1e-05):  # Метод k-means
    if type(k_or_guess) == int:
        code_book = obs[np.random.choice(range(len(obs)), k_or_guess, False)]
    else:
        code_book = k_or_guess
    prev_distortion = -1
    cur_distortion = distance.cdist(obs, code_book, 'sqeuclidean').min(axis = 1).sum()
    
    while abs(cur_distortion - prev_distortion) > thresh:
        new_code_book = np.empty((len(code_book),len(code_book[0])))
        distortion = distance.cdist(obs, code_book, 'sqeuclidean')
        temp = np.hstack((obs, distortion.argmin(axis=1).reshape(len(distortion),1)))
        for i in range(len(code_book)):
            new_code_book[i] = np.vstack(row for row in temp if row[-1] == i).mean(axis=0)[:-1]
        code_book = new_code_book
        prev_distortion = cur_distortion
        cur_distortion = distance.cdist(obs, code_book, 'sqeuclidean').min(axis = 1).sum()
        
    return code_book, cur_distortion 

In [ ]:
standardized = standardization(data[:,2:])
dist_k = np.empty(15)                          # Elbow-метод для категорий возраст-годовой доход
for i in range(1,16):
    codebook, distor = kmeans(standardized[:,[0,1]], i)
    dist_k[i-1] = distor
plt.plot(range(1,16), dist_k, 'o-r', label = "elbow method")
plt.legend()
plt.grid(True)

In [ ]:
codebook, distor = kmeans(standardized[:,[0,1]], 3)    # Найдя оптимальное k, разделяем на k кластеров(тут k=3)

distortion = distance.cdist(standardized[:,[0,1]], codebook, 'sqeuclidean')
matrix = np.hstack((standardized[:,[0,1]], distortion.argmin(axis=1).reshape(len(distortion),1)))
part1 = np.vstack(row for row in matrix if row[-1] == 0)
part2 = np.vstack(row for row in matrix if row[-1] == 1)
part3 = np.vstack(row for row in matrix if row[-1] == 2)

plt.scatter(part1[:,0], part1[:,1], c='y', label='First group')
plt.scatter(part2[:,0], part2[:,1], c='c', label='Second group')
plt.scatter(part3[:,0], part3[:,1], c='b', label='Third group')
plt.axis('equal')
plt.xlabel("Age")
plt.ylabel('Annual Income')
plt.legend()
plt.show()
# Видно, что годовой доход людей двух кластеров(молодые люди и люди старшего возраста) меньше, чем у людей
# третьего кластера(в основном люди среднего возраста) - довольно показательно.

In [ ]:
dist_k = np.empty(15)                        # Elbow-метод для категорий возраст-затраты
for i in range(1,16):
    codebook, distor = kmeans(standardized[:,[0,2]], i)
    dist_k[i-1] = distor
plt.plot(range(1,16), dist_k, 'o-r', label = "elbow method")
plt.legend()
plt.grid(True)

In [ ]:
codebook, distor = kmeans(standardized[:,[0,2]], 4)     # Найдя оптимальное k, разделяем на k кластеров(тут k=4)

distortion = distance.cdist(standardized[:,[0,2]], codebook, 'sqeuclidean')
matrix = np.hstack((standardized[:,[0,2]], distortion.argmin(axis=1).reshape(len(distortion),1)))
part1 = np.vstack(row for row in matrix if row[-1] == 0)
part2 = np.vstack(row for row in matrix if row[-1] == 1)
part3 = np.vstack(row for row in matrix if row[-1] == 2)
part4 = np.vstack(row for row in matrix if row[-1] == 3)


plt.scatter(part1[:,0], part1[:,1], c='y', label='First group')
plt.scatter(part2[:,0], part2[:,1], c='c', label='Second group')
plt.scatter(part3[:,0], part3[:,1], c='b', label='Third group')
plt.scatter(part4[:,0], part4[:,1], c='r', label='Fourth group')

plt.axis('equal')
plt.xlabel("Age")
plt.ylabel('Spending Score')
plt.legend()
plt.show()
# Люди 4го кластера(довольно молодые) выделяются как тратящие больше всего, остальные 3 кластера распределены не
# так густо, хотя в 1 кластере заметны люди среднего возраста, тратятящие тоже средне.

In [ ]:
dist_k = np.empty(15)                       # Elbow-метод для категорий годовой доход-затраты
for i in range(1,16):
    codebook, distor = kmeans(standardized[:,[1,2]], i)
    dist_k[i-1] = distor
plt.plot(range(1,16), dist_k, 'o-r', label = "elbow method")
plt.legend()
plt.grid(True)

In [ ]:
codebook, distor = kmeans(standardized[:,[1,2]], 5)   # Найдя оптимальное k, разделяем на k кластеров(тут k=5)

distortion = distance.cdist(standardized[:,[1,2]], codebook, 'sqeuclidean')
matrix = np.hstack((standardized[:,[1,2]], distortion.argmin(axis=1).reshape(len(distortion),1)))
part1 = np.vstack(row for row in matrix if row[-1] == 0)
part2 = np.vstack(row for row in matrix if row[-1] == 1)
part3 = np.vstack(row for row in matrix if row[-1] == 2)
part4 = np.vstack(row for row in matrix if row[-1] == 3)
part5 = np.vstack(row for row in matrix if row[-1] == 4)


plt.scatter(part1[:,0], part1[:,1], c='y', label='First group')
plt.scatter(part2[:,0], part2[:,1], c='c', label='Second group')
plt.scatter(part3[:,0], part3[:,1], c='b', label='Third group')
plt.scatter(part4[:,0], part4[:,1], c='r', label='Fourth group')
plt.scatter(part5[:,0], part5[:,1], c='g', label='Fifth group')


plt.axis('equal')
plt.xlabel("Annual Income")
plt.ylabel('Spending Score')
plt.legend()
plt.show()
# 5 кластеров распределены довольно равномерно. Выделяется первый кластер со средним годовым доходом и средними
# затратами - показательно.